Устанавливаем библиотеку PyQt6

In [ ]:
!pip install PyQt6

Создаем приложение

In [6]:
import ipywidgets as widgets
from IPython.display import display, Image, HTML
import base64
from PIL import Image as PILImage
from PIL import ImageDraw
import io

class ColabProfileApp:
    def __init__(self):
        self.profile_image = None
        self.setup_ui()

    def setup_ui(self):
        # Заголовок
        title = widgets.HTML("<h1 style='color: #2c3e50; text-align: center;'> Профиль студента МАИ</h1>")

        # Фото профиля
        self.photo_output = widgets.Output()
        self.draw_default_avatar()

        # Кнопка загрузки
        self.upload_btn = widgets.FileUpload(
            description=' Загрузить фото',
            multiple=False
        )
        self.upload_btn.observe(self.on_upload, names='value')

        # Информация
        info_html = widgets.HTML("""
        <div style='background: #f8f9fa; padding: 20px; border-radius: 10px;'>
            <h3 style='color: #2c3e50;'>Милена Колосовская</h3>

            <h4 style='color: #34495e;'> Биография</h4>
            <p style='color: #7f8c8d;'>Студентка Московского Авиационного Института. Обучаюсь на 2 курсе 3 факультета по специальности "Инноватика"</p>

            <h4 style='color: #34495e;'> Навыки</h4>
            <p style='color: #7f8c8d;'>Python | SQL </p>

            <h4 style='color: #34495e;'> Опыт работы</h4>
            <p style='color: #7f8c8d;'>
                <b>Курьер</b> (Сентябрь 2023 - Июнь 2024)<br>
                <b>Оператор пункта выдачи</b> (Октябрь 2025 - настоящее время)
            </p>
        </div>
        """)

        # Собираем интерфейс
        display(widgets.VBox([
            title,
            self.photo_output,
            self.upload_btn,
            info_html
        ]))

    def draw_default_avatar(self):
        with self.photo_output:
            self.photo_output.clear_output()
            display(HTML("""
            <div style='
                width: 150px;
                height: 150px;
                background: #ecf0f1;
                border-radius: 50%;
                border: 3px solid #3498db;
                display: flex;
                align-items: center;
                justify-content: center;
                font-size: 40px;
                margin: 0 auto;
            '></div>
            <p style='text-align: center; color: #95a5a6;'>Фото профиля</p>
            """))

    def on_upload(self, change):
        if not change['new']:
            return

        uploaded_file = list(change['new'].values())[0]
        image_data = uploaded_file['content']

        # Отображаем загруженное фото
        with self.photo_output:
            self.photo_output.clear_output()

            # Создаем круглое изображение
            image = PILImage.open(io.BytesIO(image_data))
            image = image.resize((150, 150))

            # Создаем круглую маску
            mask = PILImage.new('L', (150, 150), 0)
            draw = ImageDraw.Draw(mask)  # Исправлено: используем ImageDraw вместо PILImage.Draw
            draw.ellipse((0, 0, 150, 150), fill=255)

            result = PILImage.new('RGBA', (150, 150))
            result.putalpha(mask)
            result.paste(image, (0, 0))

            # Конвертируем для отображения
            buffered = io.BytesIO()
            result.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode()

            display(HTML(f"""
            <div style='text-align: center;'>
                <img src='data:image/png;base64,{img_str}' style='
                    width: 150px;
                    height: 150px;
                    border-radius: 50%;
                    border: 3px solid #3498db;
                '>
                <p style='color: #27ae60;'> Фото успешно загружено!</p>
            </div>
            """))

# Запуск приложения
app = ColabProfileApp()